### Load dependencies

In [1]:
import os
import shutil
import time
import pickle
import pandas as pd
import matplotlib.pyplot as pyplot
import librosa
import librosa.display
import gc
import numpy as np

def download_rec(uri, rec_dir):
    if not os.path.exists(rec_dir+'/'+uri.replace('/','_')):
        try:
            urllib.request.urlretrieve ("https://s3.amazonaws.com/arbimon2/"+uri, rec_dir+'/'+uri.replace('/','_'))
        except Exception as e:
            print(e)
            print('Error downloading '+uri)
        return
    

### Specify data paths:

1. train_set_dir - Folder where training data will be stored
2. recording_dir - Folder where audio recordings will be stored
3. sound_annotation_file - File storing template matching validation metadata
    

In [2]:
train_set_dir = '../data/train_tp/' # Folder where training data will be stored
recording_dir = '../recordings/' # Folder holding recordings

sound_annotation_files = ['./example_annotations.csv']
# File storing ROIs of detected sounds (animal calls) 
#     Required columns:
#          recording_id
#          species_id
#          t_min (start time of sound)
#          t_max (end time of sound)
#          uri (recording filename)

### Run remaining cells to generate training data

In [ ]:
if not os.path.exists(set_dir):
    os.mkdir(set_dir)
    for i in list(set(rois.species_id)):
        os.mkdir(set_dir+str(i)+'/')

In [3]:
if len(sound_annotation_files)==1:
    rois= pd.read_csv(sound_annotation_files)
elif len(sound_annotation_files)==0:
    print('Must provide an annotation file')
    break
elif len(sound_annotation_files)>1:
    rois = pd.read_csv(sound_annotation_files[0])
    for i in sound_annotation_files[1:]:
        tmp = pd.read_csv(sound_annotation_files[i])
        rois = pd.concat([rois,tmp])
rois.head()

,datetime,f_max,f_min,recording_id,score,songtype_id,species_id,species_name,t_max,t_min,tod,uri,validated
0,2019-04-03 14:24:00,10406.2500000000,4781.2500000000,5324133,0.3183961378,1,9741,Dendroica angelae,51.6373333333,49.2853333333,0.600000,project_1060/site_1056/2019/4/5CA4C200.flac,0
1,2019-04-14 10:15:01,10406.2500000000,4781.2500000000,5387470,0.1040763322,1,9741,Dendroica angelae,3.9306666667,1.5786666667,0.427083,project_1060/site_1037/2019/4/5CB30825.flac,0
2,2019-04-01 18:29:52,10406.2500000000,4781.2500000000,5316518,0.1718479916,1,9741,Dendroica angelae,9.5360000000,7.1840000000,0.770139,project_1060/site_1002/2019/4/5CA258A0.flac,0
3,2019-04-04 08:09:08,10406.2500000000,4781.2500000000,5438328,0.1362196428,1,9741,Dendroica angelae,36.1280000000,33.7760000000,0.339583,project_1060/site_1070/2019/4/5CA5BBA4.flac,0
4,2019-04-07 11:14:37,10406.2500000000,4781.2500000000,5342902,0.2385318273,1,9741,Dendroica angelae,52.4053333333,50.0533333333,0.468056,project_1060/site_996/2019/4/5CA9DB9D.flac,0


In [3]:
print('Number of ROIs for each species\n')

for i in list(set(rois.species_id)):
    print(str(i)+'\t\t'+str(len(rois[rois.species_id==i])))

In [4]:
window_length = 2 # sample time-window length in seconds
k = 0
t0 = time.time()
rec_loaded = False
for i in list(set(rois.recording_id)): # loop over recordings
    
    k = k+1
    if k%200==0:
        print(k)
        
    tmp = rois[rois.recording_id==i]
    audio_filename = tmp.iloc[0].uri.replace('/','_')

    for c in range(len(tmp)): # loop over spectrogram ROIs
        
        try:

            sound_start, sound_end = [tmp.iloc[c].t_min, tmp.iloc[c].t_max]
            species_id = tmp.iloc[c].species_id

            shft = ((sound_end-sound_start)-window_length)/2
            start_sample = round(sampling_rate*(sound_start+shft))
            start_sample = max(start_sample, 0)
            filename = audio_filename.split('.')[0]+'_'+str(round(start_sample/sampling_rate,2))+'-'+str(round((start_sample/sampling_rate)+window_length,2))+'.png'
            
            if not os.path.exists(set_dir+str(species_id)+'/'+filename):
                if not rec_loaded:
                    if not os.path.exists(rec_dir+'/'+audio_filename):
                        download_rec(tmp.iloc[0].uri, rec_dir)
                    try:
                        audio_data, sampling_rate = librosa.load(rec_dir+audio_filename, sr=sampling_rate)
                        rec_loaded = True
                    except Exception as e:
                        print(e)
                        continue
                S = librosa.feature.melspectrogram(y = audio_data[start_sample: start_sample+round(sampling_rate*window_length)], 
                                               sr = sampling_rate,
                                               n_fft=2048, 
                                               hop_length=512, 
                                               win_length=1024)
                dpi=100
                fig = pyplot.figure(num=None, figsize=(300/dpi, 300/dpi), dpi=dpi)
                pyplot.subplot(222)
                ax = pyplot.axes()
                ax.set_axis_off()
                librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
                pyplot.savefig(set_dir+str(species_id)+'/'+filename, bbox_inches='tight', transparent=True, pad_inches=0.0)
                pyplot.close()
                
        except Exception as e:
            print(e)
            continue
        
    rec_loaded = False    